In [ ]:
# Part I: A photo booth application
import os
import cv2
import time

# Folder to save images
save_folder = "captured_photos"

# Ensure the save directory exists
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Initialize webcam
camera = cv2.VideoCapture(0)
photo_count = 1

# Styling configurations
rect_thickness = 3
rect_line_type = cv2.LINE_AA

# Text properties
font = cv2.FONT_HERSHEY_TRIPLEX
font_size = 0.8
text_color = (255, 0, 0)  # Blue text
text_thickness = 2

# Instructions for the user
instruction_1 = "Press X to capture a photo."
instruction_2 = "Press Z to exit."

# Create a named window with a fixed size
cv2.namedWindow("Live Capture", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Live Capture", 720, 480)  # Adjusted window size

while True:
    # Capture the frame
    ret, frame = camera.read()
    if not ret:
        print("Error: Unable to capture video frame.")
        break

    # Get frame dimensions
    height, width = frame.shape[:2]
    
    # Position text dynamically
    text_pos_1 = (20, int(height * 0.1))  # 10% from top
    text_pos_2 = (20, int(height * 0.2))  # 20% from top

    # Apply text overlays
    frame = cv2.putText(frame, instruction_1, text_pos_1, font, font_size, text_color, text_thickness, rect_line_type)
    frame = cv2.putText(frame, instruction_2, text_pos_2, font, font_size, text_color, text_thickness, rect_line_type)
    
    # Display the frame
    cv2.imshow("Live Capture", frame)
    
    # Capture key input
    key = cv2.waitKey(1) & 0xFF

    # Press 'X' to start countdown and save the image
    if key == ord('x'):
        start_time = time.time()
        while time.time() - start_time < 3:
            ret, frame = camera.read()  # Keep updating the frame
            if not ret:
                break
            remaining_time = 3 - int(time.time() - start_time)
            countdown_text = f"Capturing in {remaining_time}..."
            text_size = cv2.getTextSize(countdown_text, font, font_size, text_thickness)[0]
            text_x = (width - text_size[0]) // 2
            text_y = (height - text_size[1]) // 2
            frame = cv2.putText(frame, countdown_text, (text_x, text_y), font, font_size, (0, 255, 0), text_thickness, rect_line_type)
            cv2.imshow("Live Capture", frame)
            cv2.waitKey(1)  # Keep UI responsive
        
        ret, frame = camera.read()  # Capture the latest frame for saving
        if ret:
            image_path = f"{save_folder}/photo_{photo_count}.jpg"
            cv2.imwrite(image_path, frame)
            print(f"Photo saved: {image_path}")
            photo_count += 1

    # Press 'Z' to exit
    elif key == ord('z'):
        print("Exiting application safely...")
        break

# Release resources
camera.release()
cv2.destroyAllWindows()
exit(0)


In [ ]:
### Part IIa: Brightness & Contrast
import cv2 as cv
import numpy as np

# Define folder and image path
folder_name = "captured_photos_p2"
sample_imagepath = f'{folder_name}/sample.jpg'

# Load the image
original_image = cv.imread(sample_imagepath)

# Validate if the image was loaded successfully
if original_image is None:
    print("Error: Image could not be loaded. Check the file path.")
else:
    # Display the original image
    cv.imshow("Original Image", original_image)
    cv.waitKey(500)
    
    # Adjust brightness
    brightness_factor = 150
    brightened_image = cv.add(original_image, np.ones(original_image.shape, dtype=np.uint8) * brightness_factor)
    cv.imshow("Brightened Image", brightened_image)
    cv.waitKey(500)
    
    # Adjust contrast
    contrast_factor = 0.5
    contrast_adjusted = cv.convertScaleAbs(original_image, alpha=contrast_factor, beta=0)
    cv.imshow("Contrast Adjusted Image", contrast_adjusted)
    cv.waitKey(500)
    
    # Save the results
    cv.imwrite(f"{folder_name}/brightened_image.jpg", brightened_image)
    cv.imwrite(f"{folder_name}/contrast_adjusted.jpg", contrast_adjusted)
    print("Images saved successfully!")
    
    # Wait for user input and close windows
    cv.waitKey(0)
    cv.destroyAllWindows()


In [ ]:
### Part IIb: Linear blend
import cv2 as cv
import numpy as np

# Define folder and image path
folder_name = "captured_photos_p2"

sample_imagepath1 = f'{folder_name}/sample.jpg'
sample_imagepath2 = f'{folder_name}/sample2.jpg'

# Load the images
image1 = cv.imread(sample_imagepath1)
image1 = cv.resize(image1, (500, 500))  # Resize to medium size
image2 = cv.imread(sample_imagepath2)

# Validate if images were loaded successfully
if image1 is None or image2 is None:
    print("Error: One or both images could not be loaded. Check the file paths.")
else:
    # Display the original images
    cv.imshow("Image 1", image1)
    cv.imshow("Image 2", image2)
    cv.waitKey(500)
    
    # Resize the second image to match the first image
    image2 = cv.resize(image2, (image1.shape[1], image1.shape[0]))
    
    # Ask the user for the alpha value
    alpha = float(input("Enter a blending factor (alpha) between 0 and 1: "))
    alpha = max(0, min(1, alpha))  # Ensure alpha is within range
    
    # Perform linear blending
    blended_image = cv.addWeighted(image1, 1 - alpha, image2, alpha, 0)
    
    # Display blended image
    cv.imshow("Blended Image", blended_image)
    cv.waitKey(500)
    
    # Save the blended image
    blended_image_path = f"{folder_name}/blended_image.png"
    cv.imwrite(blended_image_path, blended_image)
    print(f"Blended image saved successfully at {blended_image_path}!")
    
    # Wait for user input and close windows
    cv.waitKey(1000)
    cv.destroyAllWindows()

In [ ]:
### Part III (i) Thickness (4)
import cv2 as cv

# Get the image
folder_name = 'part3_images'
p3_image_path = f'{folder_name}/street-city-car-buildings.jpg'
image = cv.imread(p3_image_path)

if image is None:
    print("Error: Could not load the image. Check the file path.")
else:
    # Define bounding boxes for the car and pedestrian
    objects = [
        {"label": "Car", "top_left": (600, 350), "bottom_right": (720, 450)},  # Middle car
        {"label": "Pedestrian", "top_left": (250, 350), "bottom_right": (340, 540)},  # Left pedestrian
    ]

    # Draw rectangles
    for obj in objects:
        top_left = obj["top_left"]
        bottom_right = obj["bottom_right"]

        # Draw rectangle (border only)
        cv.rectangle(image, top_left, bottom_right, (0, 255, 0), 4)  # Green rectangle with thickness 4

    # Display and save the image
    cv.imshow("Rectangles with Borders", image)
    cv.imwrite("part3_images/3.1_rectangles_borders.png", image)
    print("3.1: Rectangles with borders saved successfully!")
    cv.waitKey(0)
    cv.destroyAllWindows()



In [ ]:
### Part III (ii) Thickness to (-1)
import cv2 as cv

# Get the image
folder_name = 'part3_images'
p3_image_path = f'{folder_name}/street-city-car-buildings.jpg'
image = cv.imread(p3_image_path)

if image is None:
    print("Error: Could not load the image. Check the file path.")
else:
    # Define bounding boxes for the car and pedestrian
    objects = [
        {"label": "Car", "top_left": (600, 350), "bottom_right": (720, 450)},  # Middle car
        {"label": "Pedestrian", "top_left": (250, 350), "bottom_right": (340, 540)},  # Left pedestrian
    ]

    # Draw filled rectangles
    for obj in objects:
        top_left = obj["top_left"]
        bottom_right = obj["bottom_right"]

        # Draw filled rectangle
        cv.rectangle(image, top_left, bottom_right, (0, 255, 0), -1)  # Green rectangle filled

    # Display and save the image
    cv.imshow("Filled Rectangles", image)
    cv.imwrite("part3_images/3.2_filled_rectangles.png", image)
    print("3.2: Filled rectangles saved successfully!")
    cv.waitKey(0)
    cv.destroyAllWindows()



In [ ]:
### Part III (iii) Final program (Text on rectangle)
import cv2 as cv

# Get the image
folder_name = 'part3_images'
p3_image_path = f'{folder_name}/street-city-car-buildings.jpg'
image = cv.imread(p3_image_path)

if image is None:
    print("Error: Could not load the image. Check the file path.")
else:
    # Define bounding boxes for the car and pedestrians
    objects = [
        {"label": "Car", "top_left": (600, 350), "bottom_right": (720, 450)},  # Middle car
        {"label": "Pedestrian", "top_left": (250, 350), "bottom_right": (340, 540)},  # Left pedestrian (moved up and left)
    ]

    # Draw rectangles and add labels
    for obj in objects:
        top_left = obj["top_left"]
        bottom_right = obj["bottom_right"]
        label = obj["label"]

        # Draw rectangle
        cv.rectangle(image, top_left, bottom_right, (0, 255, 0), 4)  # Green rectangle

        # Add text above the rectangle
        text_position = (top_left[0], top_left[1] - 10)
        cv.putText(image, label, text_position, cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display and save the annotated image
    cv.imshow("Car and Pedestrians Highlighted", image)
    cv.imwrite("part3_images/car_and_pedestrians_updated.png", image)
    print("Annotated image focusing on the middle car and right pedestrians saved successfully!")
    cv.waitKey(0)
    cv.destroyAllWindows()